In [1]:
%matplotlib inline

import gym
import itertools
import matplotlib
import numpy as np
import sys
import tensorflow as tf
import collections

if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.cliff_walking import CliffWalkingEnv
from lib import plotting

matplotlib.style.use('ggplot')

In [2]:
# with tf.Session() as sess:
#     print (sess)
#     if 'session' in locals() and sess is not None:
#         print('Close interactive session')
#         sess.close()
    

In [3]:
env = CliffWalkingEnv()
print ('Obs Space Dim:',env.observation_space.n)
print ('Action Space Dim:',env.action_space.n)
print ('Sample:',env.observation_space.sample())
samples = [env.observation_space.sample() for i in range(100000)]
print ('Max:',max(samples), ' Min:',min(samples))

Obs Space Dim: 48
Action Space Dim: 4
Sample: 44
Max: 47  Min: 0


In [4]:
class PolicyEstimator():
    """
    Policy Function approximator. 
    """
    
    def __init__(self, new_graph, learning_rate=0.01, scope="mody_policy_estimator"):
        with new_graph.as_default() as g:
            with g.name_scope(scope) as g1_scope:
            # with tf.variable_scope(scope):
                self.state = tf.placeholder(tf.int32, [], "state")
                self.action = tf.placeholder(dtype=tf.int32, name="action")
                self.target = tf.placeholder(dtype=tf.float32, name="target")

                # This is just table lookup estimator
                state_one_hot = tf.one_hot(self.state, int(env.observation_space.n))
                self.output_layer = tf.contrib.layers.fully_connected(
                    inputs=tf.expand_dims(state_one_hot, 0),
                    num_outputs=env.action_space.n,
                    activation_fn=None,
                    weights_initializer=tf.zeros_initializer)

                self.action_probs = tf.squeeze(tf.nn.softmax(self.output_layer))  #we get action+probs for all `env.action_space.n` states
                self.picked_action_prob = tf.gather(self.action_probs, self.action)

                # Loss and train op
                self.loss = -tf.log(self.picked_action_prob) * self.target

                self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
                self.train_op = self.optimizer.minimize(
                    self.loss, global_step=tf.contrib.framework.get_global_step())
    
    # def predict(self, state, sess=None):
    def predict(self, sess, state):
        # sess = sess or tf.get_default_session()
        # with tf.variable_scope("policy_estimator"):
        return sess.run(self.action_probs, { self.state: state })

    def update(self, state, target, action, sess=None):
        sess = sess or tf.get_default_session()
        feed_dict = { self.state: state, self.target: target, self.action: action  }
        _, loss = sess.run([self.train_op, self.loss], feed_dict)
        return loss

In [5]:
class ValueEstimator():
    """
    Value Function approximator. 
    """
    
    def __init__(self, learning_rate=0.1, scope="mody_value_estimator"):
        with tf.variable_scope(scope):
            self.state = tf.placeholder(tf.int32, [], "state")
            self.target = tf.placeholder(dtype=tf.float32, name="target")

            # This is just table lookup estimator
            state_one_hot = tf.one_hot(self.state, int(env.observation_space.n))
            self.output_layer = tf.contrib.layers.fully_connected(
                inputs=tf.expand_dims(state_one_hot, 0),
                num_outputs=1,
                activation_fn=None,
                weights_initializer=tf.zeros_initializer)

            self.value_estimate = tf.squeeze(self.output_layer)
            self.loss = tf.squared_difference(self.value_estimate, self.target)

            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            self.train_op = self.optimizer.minimize(
                self.loss, global_step=tf.contrib.framework.get_global_step())        
    
    def predict(self, state, sess=None):
        sess = sess or tf.get_default_session()
        return sess.run(self.value_estimate, { self.state: state })

    def update(self, state, target, sess=None):
        sess = sess or tf.get_default_session()
        feed_dict = { self.state: state, self.target: target }
        _, loss = sess.run([self.train_op, self.loss], feed_dict)
        return loss

In [6]:
def actor_critic(sess, env, estimator_policy, estimator_value, num_episodes, discount_factor=1.0):
    """
    Q-Learning algorithm for fff-policy TD control using Function Approximation.
    Finds the optimal greedy policy while following an epsilon-greedy policy.
    
    Args:
        env: OpenAI environment.
        estimator: Action-Value function estimator
        num_episodes: Number of episodes to run for.
        discount_factor: Lambda time discount factor.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
        epsilon_decay: Each episode, epsilon is decayed by this factor
    
    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """

    # Keeps track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))    
    
    Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
    
    for i_episode in range(num_episodes):
        # Reset the environment and pick the fisrst action
        state = env.reset()
        
        episode = []
        
        # One step in the environment
        for t in itertools.count():
            
            # Take a step
            action_probs = estimator_policy.predict(sess, state)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, reward, done, _ = env.step(action)
            
            # Keep track of the transition
            episode.append(Transition(
              state=state, action=action, reward=reward, next_state=next_state, done=done))
            
            # Update statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t
            
            # Calculate TD Target
            value_next = estimator_value.predict(next_state)
            td_target = reward + discount_factor * value_next
            td_error = td_target - estimator_value.predict(state)
            
            # Update the value estimator
            estimator_value.update(state, td_target)
            
            # Update the policy estimator
            # using the td error as our advantage estimate
            estimator_policy.update(state, td_error, action)
            
            # Print out which step we're on, useful for debugging.
            if t%10 == 0:
                print("\rStep {} @ Episode {}/{} ({})".format(
                        t, i_episode + 1, num_episodes, stats.episode_rewards[i_episode - 1]), end="")
                print ('\nStep:',t,'\tState:',state,'\tAction:',action,'\tReward:',reward)
                print ('TD Error:',td_error, '\t(',round(time.time() - t1),'s)')

            if done:
                break
                
            state = next_state
    
    return stats

In [7]:
tf.reset_default_graph()

new_graph = tf.Graph()
global_step = tf.Variable(0, name="global_step", trainable=False)
policy_estimator = PolicyEstimator(new_graph)
value_estimator = ValueEstimator()

# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
with new_graph.as_default(), tf.Session(graph = new_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print ('Session Object:', sess)
    n_episodes = 1 #300
    stats = actor_critic(sess, env, policy_estimator, value_estimator, n_episodes)

# Note, due to randomness in the policy the number of episodes you need to learn a good
# policy may vary. ~300 seemed to work well for me.
    

Session Object: <tensorflow.python.client.session.Session object at 0x000002193D02B7B8>


InternalError: Blas SGEMM launch failed : a.shape=(1, 48), b.shape=(48, 4), m=1, n=4, k=48
	 [[Node: mody_policy_estimator/fully_connected/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](mody_policy_estimator/ExpandDims, fully_connected/weights/read)]]

Caused by op 'mody_policy_estimator/fully_connected/MatMul', defined at:
  File "c:\users\prerak - nonwork\.conda\envs\py3.5\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\prerak - nonwork\.conda\envs\py3.5\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\prerak - nonwork\.conda\envs\py3.5\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "c:\users\prerak - nonwork\.conda\envs\py3.5\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\prerak - nonwork\.conda\envs\py3.5\lib\site-packages\

In [ ]:
plotting.plot_episode_stats(stats, smoothing_window=10)